# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} x_1 ~~ s.t. ~ x_1 + x_2 + x_3 = 1 ~, ~~ x \ge 0$

Dual: $\underset{\lambda \in \mathbb{R}, s \in \mathbb{R^3}}{\max} \lambda ~~ s.t. ~ [1~1~1]' \lambda + s = [1~0~0]'~, ~~s\ge0$

### 2)

## 1) Short-Step Path-Following 

In [221]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures

In [222]:
function primal_dual_diff(c, b, x, lambda)
    return abs(c'*x - b'*lambda) / (1 + abs(c'*x))
end

primal_dual_diff (generic function with 1 method)

In [223]:
"""
           STPF1(c, A, b, init_vect, max_iter, tol_step_x, eps, theta)
        Compute the optimisation of the linear problem min `c`'x, s.t. `A`x = `b` (Primal), 
        with the Short-Step Path Following method.
        
        We have the Dual problem: max `b`'λ, s.t. `A`'λ + s = c.

        `init_vect` shall be the initial vector ``(x_0,λ_0, s_0)``.
        
        `max_iter` is the maximum number of iterations.

        `tol_step_x` is the minimum tolerance of the difference between two successive iterates of x.

        `eps` is the measure of smallness.

        `theta` is the parameter such that ``||XSe- μe||_2 ≤ theta*μ ``, 

            where μ = s'x/length(x), X = Diagonal(x), S = Diagonal(s), and e = ones(1, length(x)).
       """
function STPF1(c, A, b, init_vect, max_iter, tol_step_x= 1e-4, eps = 1e-4, theta=0.4)
    x_k, lambda_k, s_k = init_vect
    n_rows, n_cols = size(A) 
    sigma = 1 - theta / sqrt(n_cols)
    k = 0 # iter
    step_x = 1
    pdd = 1
    e = ones(n_cols, 1)
    while k <= max_iter && step_x > tol_step_x && pdd > eps
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        mu_k = s_k' * x_k / n_cols
        
        # Newton's method:
        Jacob_Fk = [zeros(n_cols, n_cols) A' I(n_cols)
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk = [zeros(n_rows+n_cols, 1)
              -Xk*Sk*e + sigma*mu_k*e]
        dir_k = Jacob_Fk\Fk # error if Jacob_Fk is singular
        
        #update:
        x_k_prec = copy(x_k)
        x_k += dir_k[1:n_cols]
        lambda_k += dir_k[n_cols+1: n_cols+n_rows]
        s_k += dir_k[n_cols+n_rows+1:end]
        
        step_x = norm(x_k - x_k_prec)
        pdd = primal_dual_diff(c, b, x_k, lambda_k)
        k += 1
        
        # affichage
        println("---------------------------------------------------------------")
        println("Iter ", k, ":  step_x = ", step_x, "   pdd = ", pdd)
    end
    
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "step_x" => step_x)
end



function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

    

display_results (generic function with 1 method)

In [224]:
### probleme 1
c = [1, 0 ,0]
A = [1, 1, 1]'
b = [1];

In [225]:
# initialisation 1 
x01 = [1/3, 1/3, 1/3]
lambda01 = [-1]
s01 = [2, 1, 1]
init_vect1 = (x01, lambda01, s01);
res_opt1 = STPF1(c, A, b, init_vect1, 100);
display_results(res_opt1)

---------------------------------------------------------------
Iter 1:  step_x = 0.1632993161855452   pdd = 0.8545109914712773
---------------------------------------------------------------
Iter 2:  step_x = 0.04133143511818852   pdd = 0.6761861626203309
---------------------------------------------------------------
Iter 3:  step_x = 0.03135795982926991   pdd = 0.5317004952389174
---------------------------------------------------------------
Iter 4:  step_x = 0.028645780632299223   pdd = 0.4174698050419145
---------------------------------------------------------------
Iter 5:  step_x = 0.02585225098129478   pdd = 0.3272418274375356
---------------------------------------------------------------
Iter 6:  step_x = 0.022550398991247205   pdd = 0.2559681218751158
---------------------------------------------------------------
Iter 7:  step_x = 0.019097020740459162   pdd = 0.199744707089467
---------------------------------------------------------------
Iter 8:  step_x = 0.015803502698

OrderedDict{String,Any} with 6 entries:
  "x_opt"      => [0.000219023, 0.49989, 0.49989]
  "lambda_opt" => [-0.000438307]
  "s_opt"      => [1.00044, 0.000438307, 0.000438307]
  "n_iter"     => 29
  "pdd"        => 0.000657186
  "step_x"     => 8.05098e-5

## 2) Predictor Corrector

In [226]:
function Point_in_Neighborhood2(theta, X, S, mu, e)
    return norm(X*S*e - mu*e) <= theta*mu  # faut-il tester x>0, s>0 ?
end

function Alpha_PredictorStep(theta, x, lambda, s, dir_k, mu, e, alpha_step)
    alpha = 0
    x_k, lambda_k, s_k = x, lambda, s
    n = length(x)
    m = length(lambda)
    mu_k = s_k' * x_k / n
    while Point_in_Neighborhood2(theta, Diagonal(x_k), Diagonal(s_k), mu_k, e) == true 
        alpha += alpha_step
        x_k = x + alpha*dir_k[1:n]
        lambda_k = lambda + alpha*dir_k[n+1: n+m]
        s_k = s + alpha*dir_k[n+m+1: end]
        mu_k = s_k' * x_k / n
    end
    largest_alpha = alpha-alpha_step
    x_k = x + largest_alpha*dir_k[1:n]
    lambda_k = lambda + largest_alpha*dir_k[n+1: n+m]
    s_k = s + largest_alpha*dir_k[n+m+1: end]
    
    return x_k, lambda_k, s_k        
end

Alpha_PredictorStep (generic function with 1 method)

In [227]:
"""
           PredictorCorrector1(c, A, b, init_vect, max_iter, tol_step_x, eps, theta)
       Compute the optimisation of the linear problem min `c`'x, s.t. `A`x = `b` (Primal), 
        with the Predictor- Corrector method.
        
        We have the Dual problem: max `b`'λ, s.t. `A`'λ + s = c.

        `init_vect` shall be the initial vector ``(x_0,λ_0, s_0)``.
        
        `max_iter` is the maximum number of iterations.

        `tol_step_x` is the minimum tolerance of the difference between two successive iterates of x.

        `eps` is the measure of smallness.

        `theta1` is the parameter for the Predictor Step such that ``||XSe- μe||_2 ≤ theta*μ ``, 

            where μ = s'x/length(x), X = Diagonal(x), S = Diagonal(s), and e = ones(1, length(x)) for the inner neighborhood.

       """
function PredictorCorrector1(c, A, b, init_vect, max_iter, tol_step_x= 1e-4, eps = 1e-4, 
        theta=0.5, alpha_step=1e-3)
    x_k, lambda_k, s_k = init_vect
    n_rows, n_cols = size(A) 
    k = 0
    step_x = 1
    pdd = 1
    e = ones(n_cols, 1)
    while k <= max_iter && step_x > tol_step_x && pdd > eps
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        x_k_prec = copy(x_k)
        mu_k = s_k' * x_k / n_cols
        Jacob_Fk = [zeros(n_cols, n_cols) A' I(n_cols)
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk = [zeros(n_rows+n_cols, 1)
                -Xk*Sk*e ]
        if k%2 == 0           # predictor step, sigma=0

            dir_k = Jacob_Fk\Fk
            x_k, lambda_k, s_k = Alpha_PredictorStep(theta, x_k, lambda_k, s_k, dir_k, mu_k, e, alpha_step)
        else
            Fk += [zeros(n_rows+n_cols, 1)
                   mu_k*e ]
            dir_k = Jacob_Fk\Fk

            x_k += dir_k[1:n_cols]
            lambda_k += dir_k[n_cols+1: n_cols+n_rows]
            s_k += dir_k[n_cols+n_rows+1:end]
        end
        
        step_x = norm(x_k - x_k_prec)
        pdd = primal_dual_diff(c, b, x_k, lambda_k)
        k += 1
        
        # affichage
        println("---------------------------------------------------------------")
        println("Iter ", k, ":  step_x = ", step_x, "   pdd = ", pdd)
    end
    
    criteria = [k > max_iter, step_x <= tol_step_x, pdd <= eps]
    criteria_names = ["reached max_iter", "step_x <= tol_step_x", "pdd <= eps"]
    println("\n stopping criterion = ",criteria_names[findall(criteria)])
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "step_x" => step_x)
end

PredictorCorrector1

In [228]:
### probleme 1
c = [1, 0 ,0]
A = [1, 1, 1]'
b = [1];

In [229]:
# initialisation 1 
x01 = [1/3, 1/3, 1/3]
lambda01 = [-1]
s01 = [2, 1, 1]
init_vect1 = (x01, lambda01, s01);
res_pc1 = PredictorCorrector1(c, A, b, init_vect1, 100);
display_results(res_pc1)

---------------------------------------------------------------
Iter 1:  step_x = 0.00016329931618552725   pdd = 1.0008999100089992
---------------------------------------------------------------
Iter 2:  step_x = 0.16335818383958103   pdd = 1.1121431969456672
---------------------------------------------------------------
Iter 3:  step_x = 0.07356533103688442   pdd = 0.32078283760828497
---------------------------------------------------------------
Iter 4:  step_x = 0.05608894892105797   pdd = 0.3342085383006117
---------------------------------------------------------------
Iter 5:  step_x = 0.06765643260614837   pdd = 0.0837707524791615
---------------------------------------------------------------
Iter 6:  step_x = 0.01437411940421273   pdd = 0.0847278448647119
---------------------------------------------------------------
Iter 7:  step_x = 0.02810407133750347   pdd = 0.009186358873053224
---------------------------------------------------------------
Iter 8:  step_x = 0.0015230

OrderedDict{String,Any} with 6 entries:
  "x_opt"      => [4.61246e-5, 0.499977, 0.499977]
  "lambda_opt" => [-9.2263e-5]
  "s_opt"      => [1.00009, 9.2263e-5, 9.2263e-5]
  "n_iter"     => 10
  "pdd"        => 0.000138381
  "step_x"     => 2.23023e-5